Transformers are deep learning architectures introduced by Google in 2017 that are designed to process sequential data for downstream tasks such as translation, question answering or text summarization.

Let's first talk about text generation.
The text generation capabilities of GPT-2 is one of the most popular Transformer architectures usable by a broader audience

In [1]:
import tensorflow as tf

One of the advantages of the transformer library and a reason for its popularity is how easily we can download a specific model.

In [2]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 28.6 MB/s 
     |████████████████████████████████| 596 kB 28.2 MB/s 
     |████████████████████████████████| 6.6 MB 47.4 MB/s 
     |████████████████████████████████| 101 kB 6.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large")
GPT2 = TFGPT2LMHeadModel.from_pretrained("gpt2-large", pad_token_id=tokenizer.eos_token_id)

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.88G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


It is usually a good idea to fix the random seed to make sure the results are reproducable. As can be seen from the above result, we did download all the pretrained model from the transformers libray. It will help us to produce the code faster with more accuracy.

In [5]:
# settings

#for reproducability
SEED = 34
tf.random.set_seed(SEED)

#maximum number of words in output text
MAX_LEN = 70

The next step is we need to decode and it is one of the most important decisions when using the GPT-2 model.

We will use greedy search, the word with the highest probability is predicted as the next word in the sequence.

In [6]:
input_sequence = "I don't know about you, but there's only one thing I want to do after a long day of work"

input_sequence = "There are times when I am really tired of people, but I feel lonely too."

In [7]:
# encode context the generation is conditioned on
input_ids = tokenizer.encode(input_sequence, return_tensors='tf')

# generate text until the output length (which includes the context length) reaches 50
greedy_output = GPT2.generate(input_ids, max_length = MAX_LEN)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens = True))

Output:
----------------------------------------------------------------------------------------------------
There are times when I am really tired of people, but I feel lonely too. I feel like I'm alone in the world. I feel like I'm alone in my own body. I feel like I'm alone in my own mind. I feel like I'm alone in my own heart. I feel like I'm alone in my own mind


As can be seen from the above result, the model starts repeating itself, because the high-probability words mask the less-likely ones so they can not explore more diverse combinations.

A simple solution for that is using beam search, we can keep track of the alternative variants so that more comparisons are possible.

In [8]:
# set return_num_sequences > 1
beam_outputs = GPT2.generate(
    input_ids, 
    max_length = MAX_LEN, 
    num_beams = 5, 
    no_repeat_ngram_size = 2, 
    num_return_sequences = 5, 
    early_stopping = True
)

print('')
print("Output:\n" + 100 * '-')

# now we have 3 output sequences
for i, beam_output in enumerate(beam_outputs):
      print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))


Output:
----------------------------------------------------------------------------------------------------
0: There are times when I am really tired of people, but I feel lonely too. I don't know what to do with myself."

"I feel like I can't do anything right now," she said. "I'm so tired."
1: There are times when I am really tired of people, but I feel lonely too. I don't know what to do with myself."

"I feel like I can't do anything right now," she says. "I'm so tired."
2: There are times when I am really tired of people, but I feel lonely too. I don't know what to do with myself."

"I feel like I can't do anything right now," she says. "I'm not sure what I'm supposed to be doing with my life."
3: There are times when I am really tired of people, but I feel lonely too. I don't know what to do with myself."

"I feel like I can't do anything right now," she says. "I'm not sure what I'm supposed to be doing."
4: There are times when I am really tired of people, but I feel lonely to

As can be seen from the above result, the result is now more diverse, the message is still the same, but at least the formulations look a little different from a style point of view.

The next step is we can explore sampling- indeterministic decoding. Instead of following a strict path to find the end text with the highest probabbility, we can rather randomly pick the next word by its conditional probability distribution. This approach risks producing incoherent samplings so we can make sue of the temparature parameter which affects the probability mass distribution.

In [9]:
# Use the temperature to decrease the sensitivity to low probability candidates
sample_output = GPT2.generate(
      input_ids, 
      do_sample = True, 
      max_length = MAX_LEN, 
      top_k = 0, 
      temperature = 0.2
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens = True))

Output:
----------------------------------------------------------------------------------------------------
There are times when I am really tired of people, but I feel lonely too. I don't know why. I feel like I am alone, and I don't know why. I feel like I am alone, and I don't know why. I feel like I am alone, and I don't know why. I feel like I am


In [10]:
# What is happened when we increase the temperature?
sample_output = GPT2.generate(
      input_ids, 
      do_sample = True, 
      max_length = MAX_LEN, 
      top_k = 0, 
      temperature = 0.8
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens = True))

Output:
----------------------------------------------------------------------------------------------------
There are times when I am really tired of people, but I feel lonely too. I don't want to hug, I don't want to talk. I don't want to give hugs."

A new study published in the journal PLoS ONE suggests that loneliness is linked to increased inflammation of the immune system.

"Loneliness may trigger


This is getting more interesting, although it still feels a bit like a train of thought which is perhaps to be expected, given the content of our prompt. It is time to explore more ways to tune the output.

In Top-K sampling, the top k most likely next words are selected and the entire probability mass is shifted to these k words. So instead of increasing the chances of high probability words occurring and decreasing the chances of low probability words, we just remove the low probability words altogether.

In [11]:
#sample from only top_k most likely words
sample_output = GPT2.generate(
                             input_ids, 
                             do_sample = True, 
                             max_length = MAX_LEN, 
                             top_k = 50
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens = True), '...')

Output:
----------------------------------------------------------------------------------------------------
There are times when I am really tired of people, but I feel lonely too.

When this happens to me, I try to let it go.

I'll do my best to make this right.

The reality, however, is that it doesn't have to be that way.

In the end, I may ...


This seems like a step in the right direction. Can we do better than this?
Top-P sampling (also known as nucleus sampling) is similar to Top-K, but instead of choosing the top K most likely words we can choose the smallest set of words hose total probability is larger than p and then the enture probability mass is shifted to the words in this set. 

In [12]:
sample_output = GPT2.generate(
                             input_ids, 
                             do_sample = True, 
                             max_length = MAX_LEN, 
                             top_p = 0.8, 
                             top_k = 0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens = True), '...')

Output:
----------------------------------------------------------------------------------------------------
There are times when I am really tired of people, but I feel lonely too. I don't know where I'm going to go.

Interviewer: Have you considered moving to Mexico?

Santos: No.

Interviewer: Where are you staying now?

Santos: The house where I live ...


The main difference here is that with the Top-K sampling, the size of the set of words is static (obviously), whereas in the Top-P sampling, the size of the set cna change. To use this sampling method, we just set top_k=0 and choose the top_p value as we did shown in the above code.